In [303]:
#### Import ####

import subprocess
import reader
import tensorflow as tf

#For Bayesian LSTM cell implementation
from tensorflow.contrib.rnn import BasicLSTMCell, LSTMStateTuple, MultiRNNCell

In [348]:
#### Functions ####

def sample_random_normal(name, mean, std, shape):
    
    with tf.variable_scope("sample_random_normal"):
    
        #Inverse softplus (positive std)
        standard_dev = tf.log(tf.exp(std) - 1.0) * tf.ones(shape)
        
        mean = tf.get_variable(name + "_mean", initializer=mean, dtype=tf.float32)
        standard_deviation = tf.get_variable(name + "std", initializer=std, dtype=tf.float32)
        #Revert back to std
        standard_deviation = tf.nn.softplus(standard_deviation)
    
        #Sample standard normal
        epsilon = tf.random_normal(mean=0, stddev=1, name="epsilon", shape=shape, dtype=tf.float32)
      
        random_var = mean + standard_deviation*epsilon
    
    return random_var


#### Borrowed from PTB model ####

class PTBInput(object):
  """The input data."""

  def __init__(self, config, data, name=None):
    self.batch_size = batch_size = config.batch_size
    self.num_steps = num_steps = config.num_steps
    #self.epoch_size = ((len(data) // batch_size) - 1) // num_steps
    self.input_data, self.targets = reader.ptb_producer(
        data, batch_size, num_steps, name=name)
    

    
class TestConfig(object):
  """Tiny config, for testing."""
  init_scale = 0.1
  learning_rate = 1.0
  max_grad_norm = 1
  num_layers = 1
  num_steps = 1
  hidden_size = 2
  max_epoch = 1
  max_max_epoch = 1
  keep_prob = 1.0
  lr_decay = 0.5
  batch_size = 20
  vocab_size = 10000
  #rnn_mode = BLOCK


#### End of borrowed from PTB model ####
    

"""
    Bayesian LSTM Cell framework which inherits from tensorflows BasicLSTMCell
    
"""
class BayesianLSTMCell(BasicLSTMCell):
    def __init__(self,
                 mean, #mean for sampling weights
                 std, #standard deviation for sampling weights
                 # --- below is inherited from BasicLSTMCell --- #
                 num_units, #The number of hidden units in the cell (4*num_units)
                 forget_bias = 1.0, #bias added to forget gates
                 input_size = None, #Deprecated and unused
                 state_is_tuple = True, #If True, accepted and returned states are 2-tuples of the c_state and h_state
                 activation = tf.tanh):
        
        self.mean = mean
        self.std = std
        self.Weights = None
        self.Biases = None
        
        #From BasicLSTMCell
        super(BayesianLSTMCell, self).__init__(num_units, forget_bias, input_size, state_is_tuple, activation)
    
    #Sample weights
    def get_weights(self):
        with tf.variable_scope("CellWeights"):
            self.Weights = sample_random_normal("WeightMatrix", self.mean, self.std, shape = [6,20]) #change shape to variables!
            return self.Weights
    
    #Sample biases
    def get_biases(self):
        with tf.variable_scope("CellBiases"):
            self.Biases = sample_random_normal("BiasVector", self.mean, self.std, shape = [20]) #change shape to variable!
            return self.Biases
    
    #Object call function
    def __call__(self, inputs, state):
        with tf.variable_scope("BayesLSTMCell"):  # "BasicLSTMCell"
            
            #State is a tuple with the current cell and hidden state vectors
            cell, hidden = state
            
            all_inputs = tf.concat([inputs, hidden], 1)
            
            Weights = self.get_weights()
            Biases = self.get_biases()
            
            concat =  tf.nn.bias_add(tf.matmul(all_inputs, Weights), Biases)

            #Split data up for the 4 gates
            #i = input_gate, j = new_input, f = forget_gate, o = output_gate
            i, j, f, o = tf.split(concat, axis = 1, num_or_size_splits = 4)

            #Calculate new cell and hidden states. Calculations are as in Zaremba et al 2015
            new_cell = (cell * tf.sigmoid(f + self._forget_bias) + tf.sigmoid(i)*self._activation(j))
            new_hidden = self._activation(new_cell) * tf.sigmoid(o)
            
            #Create tuple of the new state
            new_state = LSTMStateTuple(new_cell, new_hidden)

            return new_hidden, new_state

In [349]:
#Initiate Tensorboard

subprocess.Popen(["tensorboard","--logdir=tensorboard"])

In [355]:
#### Process data ####
##
## Currently we're only interested in running a minimal sample
## To verify graphs/sessions and such
##
####

tf.reset_default_graph()

### Set initial parameters

#Current data path
path = "../data/"

### Load data
raw_data = reader.ptb_raw_data(path)
train_data, valid_data, test_data, _ = raw_data

train_input = PTBInput(config=TestConfig(), data=train_data, name="TrainInput")

#collect input data and targets by calling the ptb_producer function
#input_data, targets = reader.ptb_producer(train_data, batch_size, num_steps, name = "TrainInput")

#Embed the data: Embedding format is [vocab_size, hidden_size] = [20, 2]
embedding = tf.get_variable("embedding", [vocab_size, hidden_size], dtype=tf.float32)
#inputs = [20, 1, 2] tensor containing 20 words for one timestep with 2 hidden units
input = tf.nn.embedding_lookup(embedding, train_input.input_data)

In [356]:
#Build the graph

#Just setting some values for testing implementation

cell = BayesianLSTMCell(mean=0.0, std=1.0, num_units=hidden_size)
#cell = MultiRNNCell([cell]*2, state_is_tuple=True)
#cell = (BasicLSTMCell(num_units=hidden_size))

#cell.state_size
c = cell.zero_state(20, tf.float32)
h = cell.zero_state(20, tf.float32)
state = (c,h)

cell(inputs=input[:,0,:], state=state)

ValueError: Dimensions must be equal, but are 4 and 5 for 'BayesLSTMCell/mul' (op: 'Mul') with input shapes: [20,4], [20,5].

In [90]:
#Run a session on the graph

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    #Create event file for tensorboard
    summary_write = tf.summary.FileWriter(("tensorboard"),sess.graph)
    
    #Feed data to a placeholder
    #feed_dict = {}
    #Run desired stuff
    #Y_out = sess.run(Y, feed_dict = feed_dict)
    
#print(Y_out)